In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette
import plotly.express as px

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

s_palette = sns.cubehelix_palette(as_cmap=True)
pal = sns.color_palette("dark")
cpal = sns.cubehelix_palette(start=-0.25, rot=2, as_cmap=True)
g_palette = create_palette(8000)
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:


source_path = Path(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2025_01_31\recon")
spots_path = source_path / "out"
spots_df = pd.read_csv(spots_path / "tracked_spots.csv")
spots_df.set_index("ID", inplace=True)
save_path = spots_path
cycle_starts = np.array([0, 30, 60, 110, 160, 280])

um_per_px = 0.525
spots_df["x"] = spots_df["POSITION_X"] * um_per_px
spots_df["z"] = spots_df["POSITION_Z"] * um_per_px
spots_df["y"] = spots_df["POSITION_Y"] * um_per_px

pos = np.array(spots_df[["x", "y", "z"]])
pca = PCA(n_components=3)  # Choose the number of components you want to keep
transformed_data = pca.fit_transform(pos)
spots_df["AP"] = transformed_data[:, 0]
spots_df["theta"] = np.arctan2(transformed_data[:, 1], transformed_data[:, 2])
# fig, axes = plt.subplots(1, 3, figsize=(6, 2))
# for i in range(3):
#     axes[i].plot(pos[:, i], transformed_data[:, 0])
# plt.show()

spots_df["reflection_z"] = 2.1 * spots_df["z"].max() - spots_df["z"]
spots_df["reflection_x"] = spots_df["x"].max() - spots_df["x"]

spots_df["is_reflected"] = spots_df["x"] > spots_df["x"].max() / 2

spots_df["display_x"] = spots_df["reflection_x"] * spots_df["is_reflected"] + spots_df["x"] * (
    ~spots_df["is_reflected"])
spots_df["display_z"] = spots_df["reflection_z"] * spots_df["is_reflected"] + spots_df["z"] * (
    ~spots_df["is_reflected"])

spots_df["approx_time"] = spots_df["FRAME"] / 3.

t_grouper = spots_df.groupby("tracklet_id")["approx_time"]
spots_df["cycle_length"] = spots_df["tracklet_id"].map(t_grouper.max()) - spots_df["tracklet_id"].map(t_grouper.min())

best_spots = spots_df[spots_df["track_id"] > 0].copy()
n_tracklets = best_spots["track_id"].map(best_spots.groupby("track_id")["tracklet_id"].nunique())
best_spots = best_spots[n_tracklets > 1]

best_spots["local_density"] = 0
neighborhood_um = 30

for frame in tqdm(best_spots["FRAME"].unique()):
    frame_spots = best_spots[best_spots["FRAME"] == frame]
    coords = frame_spots[["x", "y", "z"]].values
    dists = cdist(coords, coords)
    close = dists < neighborhood_um
    local_density = close.sum(axis=1)
    best_spots.loc[frame_spots.index, "local_density"] = local_density

100%|██████████| 248/248 [00:09<00:00, 27.25it/s] 


In [18]:
import networkx as nx
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors()

frame = best_spots["FRAME"].max()
frame_spots = best_spots[best_spots["FRAME"] == frame]

pid = np.array(frame_spots.index)
tid = np.array(frame_spots["track_id"].values)
coords = frame_spots[["x", "y", "z"]].values

knn.fit(coords)
distances, indices = knn.kneighbors(coords)

pid_in = np.stack([pid]*5, -1).flatten()
tid_in = np.stack([tid]*5, -1).flatten()

pid_out = pid[np.array(indices)].flatten()
tid_out = tid[np.array(indices)].flatten()

tid_graph = nx.Graph(zip(tid_in, tid_out))
pid_graph = nx.Graph(zip(pid_in, pid_out))

In [31]:
tid_graph.remove_edges_from(nx.selfloop_edges(tid_graph))
pid_graph.remove_edges_from(nx.selfloop_edges(pid_graph))

tid_colors = nx.greedy_color(tid_graph)
print(np.max(list(tid_colors.values())))

5


In [49]:
#palette = ["#FFCBE1", "#D6E5BD", "#F9E1A8", "#BCD8EC", "#DCCcEC", "#FFDAB4", "#444444"]
palette = sns.color_palette("hls", 7)

In [38]:
uncat = [tid for tid in best_spots["track_id"].unique() if tid not in tid_colors]
tid_colors.update({tid: 6 for tid in uncat})

In [51]:
import napari
sz_start = 13
sz_end = 5
f_start = 20
f_end = 245
sz = np.linspace(sz_start, sz_end, f_end - f_start)
viewer = napari.Viewer(ndisplay=3)

tmppath = save_path / "napari_temp"
tmppath.mkdir(exist_ok=True)

for s, frame in tqdm(zip(sz, range(f_start, f_end))):
    bs = best_spots[best_spots["FRAME"] == frame]
    tids = bs["track_id"]
    coloring = [tid_colors[tid] for tid in tids]
    color = [palette[c] for c in coloring]
    l = viewer.add_points(bs[["display_x", "y", "display_z"]], name=f"frame_{frame}",
                      face_color=color, size=s, translate=[100, 0, 0])
    l.refresh()
    viewer.screenshot(str(tmppath / f"frame_{frame}.png"), flash=False)
    viewer.layers.remove(l.name)

225it [04:35,  1.22s/it]
